In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD,Adadelta,Adam
from keras.preprocessing import image
from keras.callbacks import *
from keras.applications.vgg19 import VGG19,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 64
class_num = 10

In [3]:
train_datagen = image.ImageDataGenerator(
                                         shear_range=0.3,
                                         rotation_range=15,
                                         height_shift_range=0.3,
                                         width_shift_range=0.3,
                                         channel_shift_range=15,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


In [4]:
def image_preprocess(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    return img

In [5]:
def test_model(model,test_path,csv='sample_submission.csv'):
    columns = ['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    df = pd.DataFrame(columns=columns)
    
    file_list = os.listdir(test_path)
    for i,file in enumerate(file_list):
        test_data = image_preprocess(test_path + '/' + file)
        y_pred = model.predict(test_data,batch_size=1,verbose=0)
        y_pred = np.clip(y_pred,0.001,0.999)
        y_pred = y_pred[0].tolist()
        df.loc[i] = [file] + y_pred
        
    df.to_csv(csv,index=None)

In [6]:
vgg19_base_model = VGG19(weights='imagenet', include_top=False,input_shape=(224,224,3))

x = vgg19_base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(10, activation='softmax')(x)

vgg19_model = Model(inputs=vgg19_base_model.input, outputs=x)

In [7]:
for layer in vgg19_model.layers[:22]:
    layer.trainable = False

for i,layer in enumerate(vgg19_model.layers):
    print(i,layer.name,layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_conv4 False
11 block3_pool False
12 block4_conv1 False
13 block4_conv2 False
14 block4_conv3 False
15 block4_conv4 False
16 block4_pool False
17 block5_conv1 False
18 block5_conv2 False
19 block5_conv3 False
20 block5_conv4 False
21 block5_pool False
22 global_average_pooling2d_1 True
23 dense_1 True


In [8]:
vgg19_model.compile(loss='categorical_crossentropy',
                    #optimizer=optimizers.SGD(lr=1e-3, momentum=0.9) #,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=1e-3),
                    #ptimizer=Adadelta(),
                    metrics=['accuracy'])

# fine-tune the model
vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[EarlyStopping(patience=3)])

vgg19_model.save('vgg19_GAP_fc.h5')

Epoch 1/10
312/312 [==============================] - 582s 2s/step - loss: 2.4176 - acc: 0.3592 - val_loss: 2.3856 - val_acc: 0.4215
Epoch 2/10
312/312 [==============================] - 364s 1s/step - loss: 1.0611 - acc: 0.6426 - val_loss: 1.8440 - val_acc: 0.5055
Epoch 3/10
312/312 [==============================] - 357s 1s/step - loss: 0.7828 - acc: 0.7354 - val_loss: 1.6166 - val_acc: 0.5593
Epoch 4/10
312/312 [==============================] - 359s 1s/step - loss: 0.6707 - acc: 0.7753 - val_loss: 2.0375 - val_acc: 0.4886
Epoch 5/10
312/312 [==============================] - 356s 1s/step - loss: 0.6019 - acc: 0.8062 - val_loss: 1.5532 - val_acc: 0.5636
Epoch 6/10
312/312 [==============================] - 356s 1s/step - loss: 0.5782 - acc: 0.8027 - val_loss: 1.7705 - val_acc: 0.5309
Epoch 7/10
312/312 [==============================] - 355s 1s/step - loss: 0.5454 - acc: 0.8162 - val_loss: 1.4735 - val_acc: 0.5992
Epoch 8/10
312/312 [==============================] - 353s 1s/step - 

In [9]:
# vgg19_model = load_model('vgg19_GAP_fc.h5')
    
# # for layer in vgg19_model.layers:
# #     layer.trainable = True
# for layer in vgg19_model.layers[:17]:
#     layer.trainable = False
    
# for layer in vgg19_model.layers[17:]:
#     layer.trainable = True
    
# vgg19_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.Adam(lr=1e-4),
#                     metrics=['accuracy'])  

# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
#                               patience=5, min_lr=1e-6)

# vgg19_GAP_history = vgg19_model.fit_generator(
#                         train_generator,
#                         steps_per_epoch=nb_train_samples // batch_size,
#                         epochs=50,
#                         validation_data=validation_generator,
#                         validation_steps=nb_val_samples // batch_size,
#                         callbacks=[
# #                                     TensorBoard(log_dir='./logs/vgg19_GAP'),
# #                                     CSVLogger('vgg19_GAP'), 
#                                     reduce_lr,
#                                   ])
# #                                    EarlyStopping(patience=3)])
# vgg19_model.save('vgg19_GAP_train_onetime.h5')

In [10]:
vgg19_model = load_model('vgg19_GAP_fc.h5')

for layer in vgg19_model.layers[:17]:
    layer.trainable = False
    
for layer in vgg19_model.layers[17:]:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'), 
                                   EarlyStopping(patience=3)])
vgg19_model.save('vgg19_GAP_17.h5')

Epoch 1/20
312/312 [==============================] - 364s 1s/step - loss: 0.3325 - acc: 0.9016 - val_loss: 1.1566 - val_acc: 0.7027
Epoch 2/20
312/312 [==============================] - 358s 1s/step - loss: 0.0880 - acc: 0.9733 - val_loss: 0.4795 - val_acc: 0.8788
Epoch 3/20
312/312 [==============================] - 360s 1s/step - loss: 0.0636 - acc: 0.9807 - val_loss: 0.6331 - val_acc: 0.8674
Epoch 4/20
312/312 [==============================] - 359s 1s/step - loss: 0.0489 - acc: 0.9850 - val_loss: 0.5794 - val_acc: 0.8712
Epoch 5/20
312/312 [==============================] - 357s 1s/step - loss: 0.0383 - acc: 0.9890 - val_loss: 0.9139 - val_acc: 0.7864


In [11]:
vgg19_model = load_model('vgg19_GAP_17.h5')

for layer in vgg19_model.layers[:12]:
    layer.trainable = False
    
for layer in vgg19_model.layers[12:]:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'), #])
                                   EarlyStopping(patience=3)])

vgg19_model.save('vgg19_GAP_12.h5')

Epoch 1/20
312/312 [==============================] - 384s 1s/step - loss: 0.1122 - acc: 0.9658 - val_loss: 0.4469 - val_acc: 0.8708
Epoch 2/20
312/312 [==============================] - 381s 1s/step - loss: 0.0714 - acc: 0.9786 - val_loss: 0.5198 - val_acc: 0.8597
Epoch 3/20
312/312 [==============================] - 380s 1s/step - loss: 0.0504 - acc: 0.9843 - val_loss: 0.8173 - val_acc: 0.8568
Epoch 4/20
312/312 [==============================] - 380s 1s/step - loss: 0.0523 - acc: 0.9841 - val_loss: 0.3360 - val_acc: 0.9119
Epoch 5/20
312/312 [==============================] - 380s 1s/step - loss: 0.0409 - acc: 0.9880 - val_loss: 0.6760 - val_acc: 0.8309
Epoch 6/20
312/312 [==============================] - 379s 1s/step - loss: 0.0428 - acc: 0.9874 - val_loss: 0.8387 - val_acc: 0.8453
Epoch 7/20
312/312 [==============================] - 380s 1s/step - loss: 0.0346 - acc: 0.9893 - val_loss: 0.5834 - val_acc: 0.8682


In [12]:
vgg19_model = load_model('vgg19_GAP_12.h5')

for layer in vgg19_model.layers[:7]:
    layer.trainable = False
    
for layer in vgg19_model.layers[7:]:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'), 
                                   EarlyStopping(patience=3)])

vgg19_model.save('vgg19_GAP_7.h5')

Epoch 1/20
312/312 [==============================] - 511s 2s/step - loss: 0.0617 - acc: 0.9815 - val_loss: 0.5290 - val_acc: 0.8834
Epoch 2/20
312/312 [==============================] - 507s 2s/step - loss: 0.0508 - acc: 0.9846 - val_loss: 0.5082 - val_acc: 0.8958
Epoch 3/20
312/312 [==============================] - 506s 2s/step - loss: 0.0369 - acc: 0.9899 - val_loss: 0.4148 - val_acc: 0.9153
Epoch 4/20
312/312 [==============================] - 505s 2s/step - loss: 0.0329 - acc: 0.9896 - val_loss: 0.8338 - val_acc: 0.8411
Epoch 5/20
312/312 [==============================] - 505s 2s/step - loss: 0.0329 - acc: 0.9901 - val_loss: 0.4829 - val_acc: 0.8958
Epoch 6/20
312/312 [==============================] - 504s 2s/step - loss: 0.0280 - acc: 0.9906 - val_loss: 0.4770 - val_acc: 0.9174


In [13]:
vgg19_model = load_model('vgg19_GAP_7.h5')

for layer in vgg19_model.layers:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=5e-5),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'), 
                                   EarlyStopping(patience=3)])

vgg19_model.save('vgg19_GAP_all.h5')

Epoch 1/20
312/312 [==============================] - 749s 2s/step - loss: 0.0220 - acc: 0.9945 - val_loss: 0.5028 - val_acc: 0.8961
Epoch 2/20
312/312 [==============================] - 741s 2s/step - loss: 0.0125 - acc: 0.9963 - val_loss: 0.5324 - val_acc: 0.9072
Epoch 3/20
312/312 [==============================] - 741s 2s/step - loss: 0.0191 - acc: 0.9939 - val_loss: 0.3758 - val_acc: 0.9093
Epoch 4/20
312/312 [==============================] - 739s 2s/step - loss: 0.0075 - acc: 0.9980 - val_loss: 0.9293 - val_acc: 0.8843
Epoch 5/20
312/312 [==============================] - 741s 2s/step - loss: 0.0175 - acc: 0.9945 - val_loss: 0.5516 - val_acc: 0.9013
Epoch 6/20
312/312 [==============================] - 740s 2s/step - loss: 0.0101 - acc: 0.9969 - val_loss: 0.5500 - val_acc: 0.8669


In [14]:
vgg19_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=1e-5),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'), 
                                   EarlyStopping(patience=2)])

Epoch 1/20
312/312 [==============================] - 738s 2s/step - loss: 0.0032 - acc: 0.9990 - val_loss: 0.5249 - val_acc: 0.9008
Epoch 2/20
312/312 [==============================] - 735s 2s/step - loss: 0.0013 - acc: 0.9995 - val_loss: 0.5757 - val_acc: 0.9102
Epoch 3/20
312/312 [==============================] - 735s 2s/step - loss: 0.0020 - acc: 0.9994 - val_loss: 0.5775 - val_acc: 0.9136


In [15]:
vgg19_model.save('vgg19_GAP_final.h5')

In [16]:
# vgg19_model = load_model('vgg19_GAP_final.h5')
    
# for layer in vgg19_model.layers:
#     layer.trainable = True
# # for layer in vgg19_model.layers[:17]:
# #     layer.trainable = False
    
# # for layer in vgg19_model.layers[17:]:
# #     layer.trainable = True
    
# vgg19_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.Adam(lr=1e-4),
#                     metrics=['accuracy'])  

# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                               patience=3, min_lr=1e-6)

# vgg19_GAP_history = vgg19_model.fit_generator(
#                         train_generator,
#                         steps_per_epoch=nb_train_samples // batch_size,
#                         epochs=10,
#                         validation_data=validation_generator,
#                         validation_steps=nb_val_samples // batch_size,
#                         callbacks=[
# #                                     TensorBoard(log_dir='./logs/vgg19_GAP'),
# #                                     CSVLogger('vgg19_GAP'), 
#                                     reduce_lr,
#                                   ])
# #                                    EarlyStopping(patience=3)])
# # vgg19_model.save('vgg19_GAP_train_onetime.h5')

In [17]:
# vgg19_model.save('vgg19_GAP_final.h5')